# GOLD - Publicación del Modelo Estrella (Dimensiones Tipo I + Fact)

**Objetivo.** Publicar el modelo estrella en GOLD cumpliendo la consigna:
- Dimensión de **tiempo** (definición libre).
- Dimensiones de negocio **Tipo I** (se sobrescriben valores; no se duplican al reprocesar).
- **Tabla Fact** con tipos de datos específicos y FK a las dimensiones.
- Verificaciones de consistencia.

**Entradas**
- `silver.iowa_sales_clean` (curada y tipada)

**Salidas**
- `gold.dim_time` (Tipo I)
- `gold.dim_store` (Tipo I)
- `gold.dim_item`  (Tipo I)
- `gold.fact_sales` (Delta, `PARTITIONED BY (year, month)`)

**Tareas que realiza**
1. **`dim_time` (Tipo I)**  
   - Calcula `MIN/MAX sale_date` desde SILVER para generar el rango real de fechas.  
   - Deriva atributos: `date_key (yyyymmdd)`, `year`, `quarter`, `month`, `month_name`, `day`, `week_of_year`, `day_of_week`, `day_name`, `is_weekend`.  
   - **Carga idempotente** con `MERGE ON date_key` (no duplica al reprocesar).

2. **`dim_store` (Tipo I)**  
   - Una sola fila por `store_id` (toma la versión más reciente con `ROW_NUMBER` por `sale_date/ingestion_ts`).  
   - `zipcode` como `STRING` (evitar pérdida de ceros).  
   - **MERGE ON store_id**.

3. **`dim_item` (Tipo I)**  
   - Una sola fila por `item_no` (versión más reciente).  
   - Tipos específicos (`INT`/`STRING` según columna).  
   - **MERGE ON item_no**.

4. **`fact_sales` (tipos específicos + FK)**  
   - Une SILVER con las dimensiones (`date_key`, `store_id`, `item_no`).  
   - Mantiene medidas: `sale_bottles (INT)`, `sale_dollars (DECIMAL 14,2)`, `sale_liters/gallons (DECIMAL 14,3)`, `state_bottle_cost/retail (DECIMAL 12,2)`.  
   - **Idempotente** con `MERGE ON invoice_line_no`.  
   - **Deduplicación del source** previa al MERGE para evitar error de “multiple source rows match”.

**Idempotencia y Tipo I**
- Todas las dimensiones usan `MERGE` por su clave natural (ej. `store_id`, `item_no`, `date_key`):
  - `WHEN MATCHED THEN UPDATE` (sobrescribe, no guarda historia)  
  - `WHEN NOT MATCHED THEN INSERT`  
- Reejecutar **no** genera duplicados.

**Verificaciones (QA)**
- Conteo de filas en cada dimensión (`COUNT(*)`).  
- **Consistencia de fact**:  
  - `rows_joinable_via_dims` (SILVER que se une a TODAS las dimens) **=** `rows_fact_loaded`.  
  - Diferencia con SILVER = filas con llaves nulas/no mapeables (esperable).

**Evidencias esperadas (para 2025-10)**
- `dim_time`: ~4.897 filas (2012-01-03 → 2025-05-30).  
- `dim_store`: ~3.238 filas (primer load).  
- `dim_item`: ~10.591 filas (primer load).  
- `fact_sales` (2025-10): ~11,031,998 (coincide con “joinable”).

**Notas**
- El notebook está diseñado para correr en **Jobs** como parte de la orquestación (Parte 5).  
- Separar “carga de dimensiones” y “carga de fact” en celdas claras facilita reintentos.


In [0]:
# --- Widgets estándar (parámetros globales) ---
dbutils.widgets.text("catalog", "ct_andresolguin_finalproject")
dbutils.widgets.text("schema",  "gold")  
dbutils.widgets.text("bronze_schema", "bronze")
dbutils.widgets.text("silver_schema", "silver")
dbutils.widgets.text("gold_schema",   "gold")

catalog = dbutils.widgets.get("catalog")
schema  = dbutils.widgets.get("schema")
bronze  = dbutils.widgets.get("bronze_schema")
silver  = dbutils.widgets.get("silver_schema")
gold    = dbutils.widgets.get("gold_schema")

try:
    cur_cat = spark.sql("select current_catalog()").first()[0]
    cur_sch = spark.sql("select current_schema()").first()[0]
except Exception:
    cur_cat, cur_sch = None, None

if (cur_cat, cur_sch) != (catalog, schema):
    spark.sql(f"USE CATALOG `{catalog}`")
    spark.sql(f"USE `{schema}`")

print(f"Contexto activo: {catalog}.{schema}  |  bronze={bronze}, silver={silver}, gold={gold}")


Contexto activo: ct_andresolguin_finalproject.silver  |  bronze=bronze, silver=silver, gold=gold


In [0]:
%sql
-- REBUILD completo de GOLD desde la silver estricta
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

CREATE OR REPLACE TABLE fact_sales
USING DELTA
PARTITIONED BY (year, month)
AS
SELECT *
FROM ct_andresolguin_finalproject.silver.iowa_clean_v2_strict;


num_affected_rows,num_inserted_rows


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

/* GOLD: fact construida únicamente con SILVER estricto */
CREATE OR REPLACE TABLE fact_sales
USING DELTA
PARTITIONED BY (year, month)
AS
SELECT *
FROM ct_andresolguin_finalproject.silver.iowa_clean_strict;


num_affected_rows,num_inserted_rows


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

SHOW TABLES LIKE 'fact_sales';


database,tableName,isTemporary
gold,fact_sales,false


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH gold_2025_10 AS (
  SELECT * FROM fact_sales WHERE year = 2025 AND month = 10
),
inv_gold AS (
  SELECT DISTINCT regexp_extract(invoice_line_no, '^[0-9]+', 0) AS invoice_id
  FROM gold_2025_10
  WHERE invoice_line_no IS NOT NULL AND invoice_line_no <> ''
),
inv_rej AS (
  SELECT DISTINCT regexp_extract(invoice_line_no, '^[0-9]+', 0) AS invoice_id
  FROM ct_andresolguin_finalproject.silver.iowa_rejected
  WHERE year = 2025 AND month = 10
    AND invoice_line_no IS NOT NULL AND invoice_line_no <> ''
),
overlap AS (
  SELECT COUNT(*) AS invoices_in_both
  FROM inv_gold g JOIN inv_rej r USING (invoice_id)
)
SELECT 'gold_rows_2025_10' AS metric, COUNT(*) AS n FROM gold_2025_10
UNION ALL
SELECT 'gold_invoices_2025_10', COUNT(*) FROM inv_gold
UNION ALL
SELECT 'invoices_overlap_gold_vs_rejected', invoices_in_both FROM overlap;


metric,n
gold_invoices_2025_10,76
invoices_overlap_gold_vs_rejected,0
gold_rows_2025_10,728


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;

-- 1) Silver estricto: excluir cualquier FACTURA presente en rejected (por mes evaluado)
CREATE OR REPLACE VIEW silver.iowa_clean_strict AS
SELECT c.*
FROM silver.iowa_clean c
LEFT ANTI JOIN (
  SELECT DISTINCT regexp_extract(invoice_line_no,'^[0-9]+',0) AS invoice_id
  FROM silver.iowa_rejected
  WHERE year = 2025 AND month = 10
) r
ON regexp_extract(c.invoice_line_no,'^[0-9]+',0) = r.invoice_id
WHERE c.year = 2025 AND c.month = 10;

-- 2) GOLD solo desde el estricto
USE SCHEMA gold;
CREATE OR REPLACE TABLE fact_sales
USING DELTA
PARTITIONED BY (year, month)
AS
SELECT * FROM ct_andresolguin_finalproject.silver.iowa_clean_strict;

-- 3) Verificación: el profe-test (esperado: overlap = 0)
WITH gold_inv AS (
  SELECT DISTINCT regexp_extract(invoice_line_no,'^[0-9]+',0) inv
  FROM gold.fact_sales WHERE year=2025 AND month=10
),
rej_inv AS (
  SELECT DISTINCT regexp_extract(invoice_line_no,'^[0-9]+',0) inv
  FROM silver.iowa_rejected WHERE year=2025 AND month=10
),
overlap AS (
  SELECT COUNT(*) n FROM gold_inv g JOIN rej_inv r USING (inv)
),
gc AS (SELECT COUNT(*) n FROM gold.fact_sales WHERE year=2025 AND month=10),
sc AS (SELECT COUNT(*) n FROM silver.iowa_clean_strict WHERE year=2025 AND month=10)
SELECT 'gold_rows_2025_10' AS metric, (SELECT n FROM gc) AS n
UNION ALL
SELECT 'clean_strict_rows_2025_10', (SELECT n FROM sc)
UNION ALL
SELECT 'invoices_overlap_gold_vs_rejected_2025_10', (SELECT n FROM overlap);


metric,n
gold_rows_2025_10,728
clean_strict_rows_2025_10,728
invoices_overlap_gold_vs_rejected_2025_10,0


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;

CREATE OR REPLACE VIEW gold.qa_guardrail_2025_10 AS
WITH gold_inv AS (
  SELECT DISTINCT regexp_extract(invoice_line_no,'^[0-9]+',0) inv
  FROM gold.fact_sales WHERE year=2025 AND month=10
),
rej_inv AS (
  SELECT DISTINCT regexp_extract(invoice_line_no,'^[0-9]+',0) inv
  FROM silver.iowa_rejected WHERE year=2025 AND month=10
),
overlap AS (
  SELECT COUNT(*) n FROM gold_inv g JOIN rej_inv r USING (inv)
),
gc AS (SELECT COUNT(*) n FROM gold.fact_sales WHERE year=2025 AND month=10),
sc AS (SELECT COUNT(*) n FROM silver.iowa_clean_strict WHERE year=2025 AND month=10),
rc AS (SELECT COUNT(*) n FROM silver.iowa_rejected     WHERE year=2025 AND month=10)
SELECT 'gold_rows'      AS metric, (SELECT n FROM gc) AS n UNION ALL
SELECT 'clean_strict'   AS metric, (SELECT n FROM sc) AS n UNION ALL
SELECT 'rejected_rows'  AS metric, (SELECT n FROM rc) AS n UNION ALL
SELECT 'invoices_overlap_gold_vs_rejected' AS metric, (SELECT n FROM overlap) AS n;


In [0]:
# Parámetros (widgets) para GOLD
def w(name, default):
    try: dbutils.widgets.text(name, default)
    except: pass

w("catalog", "ct_andresolguin_finalproject")
w("schema_gold", "gold")
w("process_date", "20251018")  # AAAAMMDD

catalog      = dbutils.widgets.get("catalog")
schema_gold  = dbutils.widgets.get("schema_gold")
process_date = dbutils.widgets.get("process_date").strip()

# Derivados para filtros (si los quisieras usar en la FACT)
yyyy, mm = int(process_date[:4]), int(process_date[4:6])

# Contexto
spark.sql(f"USE CATALOG {catalog}")

print(f"[GOLD] catalog={catalog} | schema={schema_gold} | process_date={process_date} (year={yyyy}, month={mm})")


[GOLD] catalog=ct_andresolguin_finalproject | schema=gold | process_date=20251018 (year=2025, month=10)


In [0]:
%sql
SELECT
  MIN(sale_date) AS min_date,
  MAX(sale_date) AS max_date
FROM ct_andresolguin_finalproject.silver.iowa_sales_clean
WHERE sale_date IS NOT NULL;


min_date,max_date
2012-01-03,2025-05-30


## Paso 1 — Dimensión de tiempo (CREATE/MERGE)
- Crea `gold.dim_time` si no existe.  
- Genera fechas con atributos (año, mes, semana, día, etc.).  
- MERGE tipo I para idempotencia.


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
CREATE SCHEMA IF NOT EXISTS gold;

-- 1) Tabla destino (si no existe)
CREATE TABLE IF NOT EXISTS ct_andresolguin_finalproject.gold.dim_time (
  date_key      INT,
  date          DATE,
  year          INT,
  quarter       INT,
  month         INT,
  month_name    STRING,
  day           INT,
  week_of_year  INT,
  day_of_week   INT,   -- 1=Dom, 7=Sáb (Spark)
  day_name      STRING,
  is_weekend    BOOLEAN
) USING DELTA;

-- 2) Rango dinámico desde SILVER + atributos de fecha
WITH bounds AS (
  SELECT
    MIN(sale_date) AS min_date,
    MAX(sale_date) AS max_date
  FROM ct_andresolguin_finalproject.silver.iowa_sales_clean
  WHERE sale_date IS NOT NULL
),
dates AS (
  SELECT explode(sequence(min_date, max_date, interval 1 day)) AS d
  FROM bounds
),
src AS (
  SELECT
    CAST(date_format(d,'yyyyMMdd') AS INT) AS date_key,
    d                                      AS date,
    YEAR(d)                                AS year,
    QUARTER(d)                             AS quarter,
    MONTH(d)                               AS month,
    date_format(d,'MMMM')                  AS month_name,
    DAY(d)                                 AS day,
    weekofyear(d)                          AS week_of_year,
    dayofweek(d)                           AS day_of_week,
    date_format(d,'EEEE')                  AS day_name,
    CASE WHEN dayofweek(d) IN (1,7) THEN TRUE ELSE FALSE END AS is_weekend
  FROM dates
)
-- 3) MERGE Tipo I (sin duplicados al reprocesar)
MERGE INTO ct_andresolguin_finalproject.gold.dim_time AS tgt
USING src
ON tgt.date_key = src.date_key
WHEN MATCHED THEN UPDATE SET
  tgt.date         = src.date,
  tgt.year         = src.year,
  tgt.quarter      = src.quarter,
  tgt.month        = src.month,
  tgt.month_name   = src.month_name,
  tgt.day          = src.day,
  tgt.week_of_year = src.week_of_year,
  tgt.day_of_week  = src.day_of_week,
  tgt.day_name     = src.day_name,
  tgt.is_weekend   = src.is_weekend
WHEN NOT MATCHED THEN INSERT (
  date_key, date, year, quarter, month, month_name, day, week_of_year, day_of_week, day_name, is_weekend
) VALUES (
  src.date_key, src.date, src.year, src.quarter, src.month, src.month_name, src.day, src.week_of_year, src.day_of_week, src.day_name, src.is_weekend
);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4897,4897,0,0


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
CREATE SCHEMA IF NOT EXISTS gold;

-- 1) Definición destino (Delta)
CREATE TABLE IF NOT EXISTS ct_andresolguin_finalproject.gold.dim_store (
  store_id       INT,
  store_name     STRING,
  address        STRING,
  city           STRING,
  zipcode        STRING,   -- string para no perder ceros a la izquierda
  county_number  INT,
  county         STRING,
  updated_at     TIMESTAMP
) USING DELTA;

-- 2) Canonicalización (una fila por store_id, tomando la más reciente)
WITH src_raw AS (
  SELECT
    s.store_id,
    s.store_name,
    s.address,
    s.city,
    s.zipcode,
    s.county_number,
    s.county,
    s.sale_date,
    s.ingestion_ts,
    ROW_NUMBER() OVER (
      PARTITION BY s.store_id
      ORDER BY s.sale_date DESC, s.ingestion_ts DESC
    ) AS rn
  FROM ct_andresolguin_finalproject.silver.iowa_sales_clean s
  WHERE s.store_id IS NOT NULL
),
src AS (
  SELECT
    store_id, store_name, address, city, zipcode, county_number, county,
    current_timestamp() AS updated_at
  FROM src_raw
  WHERE rn = 1
)

-- 3) MERGE Tipo I (sin duplicar al reprocesar)
MERGE INTO ct_andresolguin_finalproject.gold.dim_store AS tgt
USING src
ON tgt.store_id = src.store_id
WHEN MATCHED THEN UPDATE SET
  tgt.store_name    = src.store_name,
  tgt.address       = src.address,
  tgt.city          = src.city,
  tgt.zipcode       = src.zipcode,
  tgt.county_number = src.county_number,
  tgt.county        = src.county,
  tgt.updated_at    = current_timestamp()
WHEN NOT MATCHED THEN INSERT (
  store_id, store_name, address, city, zipcode, county_number, county, updated_at
) VALUES (
  src.store_id, src.store_name, src.address, src.city, src.zipcode, src.county_number, src.county, src.updated_at
);

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3238,3238,0,0


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
CREATE SCHEMA IF NOT EXISTS gold;

-- 1) Definición destino (Delta)
CREATE TABLE IF NOT EXISTS ct_andresolguin_finalproject.gold.dim_item (
  item_no          INT,
  item_desc        STRING,
  pack             INT,
  bottle_volume_ml INT,
  category_id      INT,
  category_name    STRING,
  vendor_no        INT,
  vendor_name      STRING,
  updated_at       TIMESTAMP
) USING DELTA;

-- 2) Canonicalización (una fila por item_no reciente)
WITH src_raw AS (
  SELECT
    s.item_no,
    s.item_desc,
    s.pack,
    s.bottle_volume_ml,
    s.category_id,
    s.category_name,
    s.vendor_no,
    s.vendor_name,
    s.sale_date,
    s.ingestion_ts,
    ROW_NUMBER() OVER (
      PARTITION BY s.item_no
      ORDER BY s.sale_date DESC, s.ingestion_ts DESC
    ) AS rn
  FROM ct_andresolguin_finalproject.silver.iowa_sales_clean s
  WHERE s.item_no IS NOT NULL
),
src AS (
  SELECT
    item_no, item_desc, pack, bottle_volume_ml,
    category_id, category_name, vendor_no, vendor_name,
    current_timestamp() AS updated_at
  FROM src_raw
  WHERE rn = 1
)

-- 3) MERGE Tipo I (idempotente)
MERGE INTO ct_andresolguin_finalproject.gold.dim_item AS tgt
USING src
ON tgt.item_no = src.item_no
WHEN MATCHED THEN UPDATE SET
  tgt.item_desc         = src.item_desc,
  tgt.pack              = src.pack,
  tgt.bottle_volume_ml  = src.bottle_volume_ml,
  tgt.category_id       = src.category_id,
  tgt.category_name     = src.category_name,
  tgt.vendor_no         = src.vendor_no,
  tgt.vendor_name       = src.vendor_name,
  tgt.updated_at        = src.updated_at
WHEN NOT MATCHED THEN INSERT (
  item_no, item_desc, pack, bottle_volume_ml,
  category_id, category_name, vendor_no, vendor_name, updated_at
) VALUES (
  src.item_no, src.item_desc, src.pack, src.bottle_volume_ml,
  src.category_id, src.category_name, src.vendor_no, src.vendor_name, src.updated_at
);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10591,10591,0,0


In [0]:
%sql
-- GOLD publish (rebuild completo desde la silver estricta v2)
USE CATALOG ct_andresolguin_finalproject;
CREATE SCHEMA IF NOT EXISTS gold;
USE SCHEMA gold;

CREATE OR REPLACE TABLE ct_andresolguin_finalproject.gold.fact_sales
USING DELTA
PARTITIONED BY (year, month)
AS
SELECT *
FROM ct_andresolguin_finalproject.silver.iowa_clean_v2_strict;



num_affected_rows,num_inserted_rows


## Paso 2 — Hechos (fact_sales) desde Silver
- Joins a dimensiones requeridas (tiempo, ítem, tienda).  
- Deduplicación (`ROW_NUMBER`) por `invoice_line_no`.  
- MERGE a `gold.fact_sales` (particionado por `year`, `month`).


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

SELECT
  COUNT(*)                                   AS rows,
  MIN(sale_date)                             AS min_date,
  MAX(sale_date)                             AS max_date,
  COUNT(DISTINCT year)                       AS distinct_years,
  COUNT(DISTINCT CONCAT(year,'-',lpad(month,2,'0'))) AS distinct_year_months
FROM fact_sales;


rows,min_date,max_date,distinct_years,distinct_year_months
108224000,2012-01-03,2025-05-23,14,143


In [0]:
%sql
SHOW PARTITIONS fact_sales;
-- (si preferís ver metadatos del Delta)
-- DESCRIBE DETAIL fact_sales;


year,month
2019,3
2012,9
2016,3
2020,5
2022,7
2019,7
2021,10
2015,7
2012,4
2020,9


In [0]:
%sql
SELECT COUNT(*) AS rows_2025_10
FROM gold.fact_sales
WHERE year=2025 AND month=10;


In [0]:
%sql
WITH s AS (
  SELECT * 
  FROM ct_andresolguin_finalproject.silver.iowa_sales_clean 
  WHERE year = 2025 AND month = 10
),
joinable AS (
  SELECT s.invoice_line_no
  FROM s
  JOIN ct_andresolguin_finalproject.gold.dim_time  dt ON dt.date     = s.sale_date
  JOIN ct_andresolguin_finalproject.gold.dim_store ds ON ds.store_id = s.store_id
  JOIN ct_andresolguin_finalproject.gold.dim_item  di ON di.item_no  = s.item_no
),
fact_ct AS (
  SELECT COUNT(*) AS c_fact 
  FROM ct_andresolguin_finalproject.gold.fact_sales 
  WHERE year = 2025 AND month = 10
),
silver_ct AS (
  SELECT COUNT(*) AS c_silver FROM s
),
joinable_ct AS (
  SELECT COUNT(*) AS c_joinable FROM joinable
)
SELECT 
  c_silver   AS rows_silver_oct25,
  c_joinable AS rows_joinable_via_dims,
  c_fact     AS rows_fact_loaded
FROM silver_ct, joinable_ct, fact_ct;


rows_silver_oct25,rows_joinable_via_dims,rows_fact_loaded
15299000,11031998,11031998


In [0]:
%sql
WITH s AS (
  SELECT * 
  FROM ct_andresolguin_finalproject.silver.iowa_sales_clean 
  WHERE year = 2025 AND month = 10
),
joinable AS (
  SELECT s.invoice_line_no
  FROM s
  JOIN ct_andresolguin_finalproject.gold.dim_time  dt ON dt.date     = s.sale_date
  JOIN ct_andresolguin_finalproject.gold.dim_store ds ON ds.store_id = s.store_id
  JOIN ct_andresolguin_finalproject.gold.dim_item  di ON di.item_no  = s.item_no
),
fact_ct AS (
  SELECT COUNT(*) AS c_fact 
  FROM ct_andresolguin_finalproject.gold.fact_sales 
  WHERE year = 2025 AND month = 10
),
silver_ct AS (
  SELECT COUNT(*) AS c_silver FROM s
),
joinable_ct AS (
  SELECT COUNT(*) AS c_joinable FROM joinable
)
SELECT 
  c_silver   AS rows_silver_oct25,
  c_joinable AS rows_joinable_via_dims,
  c_fact     AS rows_fact_loaded
FROM silver_ct, joinable_ct, fact_ct;

rows_silver_oct25,rows_joinable_via_dims,rows_fact_loaded
15299000,11031998,11031998


In [0]:
%sql
-- Dimensiones: cardinalidad y unicidad de clave
WITH
dtime AS (SELECT COUNT(*) c FROM ct_andresolguin_finalproject.gold.dim_time),
dstore AS (SELECT COUNT(*) c, COUNT(DISTINCT store_id) u FROM ct_andresolguin_finalproject.gold.dim_store),
ditem  AS (SELECT COUNT(*) c, COUNT(DISTINCT item_no)  u FROM ct_andresolguin_finalproject.gold.dim_item),
fact   AS (SELECT COUNT(*) c FROM ct_andresolguin_finalproject.gold.fact_sales WHERE year=2025 AND month=10),

-- SILVER del mes
s AS (SELECT * FROM ct_andresolguin_finalproject.silver.iowa_sales_clean WHERE year=2025 AND month=10),

-- Filas “unibles” a TODAS las dims (debe igualar a fact)
joinable AS (
  SELECT s.invoice_line_no
  FROM s
  JOIN ct_andresolguin_finalproject.gold.dim_time  dt ON dt.date     = s.sale_date
  JOIN ct_andresolguin_finalproject.gold.dim_store ds ON ds.store_id = s.store_id
  JOIN ct_andresolguin_finalproject.gold.dim_item  di ON di.item_no  = s.item_no
),

-- Duplicados en claves de dimensiones (deben ser 0)
dup_store AS (
  SELECT COUNT(*) dups FROM (
    SELECT store_id FROM ct_andresolguin_finalproject.gold.dim_store GROUP BY store_id HAVING COUNT(*)>1
  )
),
dup_item AS (
  SELECT COUNT(*) dups FROM (
    SELECT item_no FROM ct_andresolguin_finalproject.gold.dim_item GROUP BY item_no HAVING COUNT(*)>1
  )
)

SELECT
  (SELECT c FROM dtime)  AS dim_time_rows,
  (SELECT c FROM dstore) AS dim_store_rows, (SELECT u FROM dstore) AS dim_store_unique_keys,
  (SELECT c FROM ditem)  AS dim_item_rows,  (SELECT u FROM ditem)  AS dim_item_unique_keys,
  (SELECT c FROM fact)   AS fact_rows_oct25,
  (SELECT COUNT(*) FROM joinable) AS joinable_rows_oct25,
  (SELECT dups FROM dup_store) AS dup_store_keys,
  (SELECT dups FROM dup_item)  AS dup_item_keys;


dim_time_rows,dim_store_rows,dim_store_unique_keys,dim_item_rows,dim_item_unique_keys,fact_rows_oct25,joinable_rows_oct25,dup_store_keys,dup_item_keys
4897,3238,3238,10591,10591,11031998,11031998,0,0


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

/* Repoblar GOLD desde el silver estricto (819 filas esperadas) */
CREATE OR REPLACE TABLE fact_sales
USING DELTA
PARTITIONED BY (year, month)
AS
SELECT * FROM ct_andresolguin_finalproject.silver.iowa_clean_strict;

/* Verificación rápida */
WITH gc AS (
  SELECT COUNT(*) n FROM gold.fact_sales WHERE year=2025 AND month=10
),
sc AS (
  SELECT COUNT(*) n FROM ct_andresolguin_finalproject.silver.iowa_clean_strict
),
g_inv AS (
  SELECT DISTINCT regexp_extract(invoice_line_no,'^[0-9]+',0) inv
  FROM gold.fact_sales WHERE year=2025 AND month=10
),
r_inv AS (
  SELECT DISTINCT regexp_extract(invoice_line_no,'^[0-9]+',0) inv
  FROM ct_andresolguin_finalproject.silver.iowa_rejected WHERE year=2025 AND month=10
),
overlap AS (SELECT COUNT(*) n FROM g_inv g JOIN r_inv r USING (inv))
SELECT 'gold_rows_2025_10' AS metric, (SELECT n FROM gc) AS n
UNION ALL
SELECT 'clean_strict_rows_total', (SELECT n FROM sc)
UNION ALL
SELECT 'invoices_overlap_gold_vs_rejected_2025_10', (SELECT n FROM overlap);


metric,n
gold_rows_2025_10,819
clean_strict_rows_total,819
invoices_overlap_gold_vs_rejected_2025_10,0


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

-- Rango y cantidad de fechas de venta reales
SELECT 
  COUNT(*)                            AS rows_total,
  MIN(CAST(date AS DATE))             AS min_sale_date,
  MAX(CAST(date AS DATE))             AS max_sale_date,
  COUNT(DISTINCT CAST(date AS DATE))  AS distinct_sale_days
FROM fact_sales;

-- Qué particiones (year, month) tiene la tabla (son del proceso)
SELECT year, month, COUNT(*) AS rows
FROM fact_sales
GROUP BY year, month
ORDER BY year, month;


year,month,rows
2025,10,910


In [0]:
%sql
-- Paso único: Diagnóstico de fechas en GOLD para 2025-10
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

SELECT
  COUNT(*)                                   AS rows_2025_10,
  COUNT(date)                                AS non_null_dates,      -- cuántas filas tienen fecha
  (COUNT(*) - COUNT(date))                   AS null_dates,          -- cuántas NO tienen fecha
  COUNT(DISTINCT date)                       AS distinct_days,
  MIN(date)                                  AS min_date,
  MAX(date)                                  AS max_date
FROM fact_sales
WHERE year = 2025 AND month = 10;


rows_2025_10,non_null_dates,null_dates,distinct_days,min_date,max_date
910,910,0,4,2012-02-29T00:00:00.000,2012-10-17T00:00:00.000


In [0]:
%sql
-- Catálogo y esquema
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

-- Vista base para queries de negocio (DIARIA / MENSUAL / ANUAL)
CREATE OR REPLACE VIEW v_sales_base AS
SELECT
  CAST(date AS DATE)          AS dte,         -- fecha del pedido (desde SILVER estricto)
  year,                                      -- ya viene en strict
  month,                                     -- ya viene en strict
  sale_dollars
FROM ct_andresolguin_finalproject.silver.iowa_clean_strict
WHERE date IS NOT NULL;                       -- clave: quedarnos solo con las filas que traen fecha válida


In [0]:
%sql
-- ¿Qué rango de fechas tenemos ahora?
SELECT COUNT(*) AS rows_total, MIN(dte) AS min_dte, MAX(dte) AS max_dte
FROM v_sales_base;

-- ¿Cuántas filas con fecha tenemos en 2025-10?
SELECT year, month, COUNT(*) AS rows_con_fecha
FROM v_sales_base
WHERE year = 2025 AND month = 10
GROUP BY year, month;


year,month,rows_con_fecha
2025,10,910


In [0]:
%sql
-- Catálogo y esquema
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

-- Tendencia diaria (últimos 30 días) desde la vista base con fecha válida
WITH lim AS (
  SELECT MAX(dte) AS max_dte FROM v_sales_base
),
last_30 AS (
  SELECT b.dte, b.sale_dollars
  FROM v_sales_base b
  JOIN lim ON 1 = 1
  WHERE b.dte >= date_sub(lim.max_dte, 30)
),
agg AS (
  SELECT
    dte,
    SUM(sale_dollars) AS sales_usd
  FROM last_30
  GROUP BY dte
)
SELECT
  dte,
  sales_usd,
  LAG(sales_usd) OVER (ORDER BY dte) AS prev_day_sales,
  ROUND(
    (sales_usd - LAG(sales_usd) OVER (ORDER BY dte))
    / NULLIF(LAG(sales_usd) OVER (ORDER BY dte), 0) * 100, 2
  ) AS dod_growth_pct
FROM agg
ORDER BY dte;


dte,sales_usd,prev_day_sales,dod_growth_pct
2012-10-17,14992.200000000004,null,null


In [0]:
%sql
-- Tendencia diaria (últimos 30 días) tomando la partición 2025-10 en GOLD
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH d AS (
  SELECT
    CAST(date AS DATE) AS dte,         -- columna correcta
    sale_dollars
  FROM fact_sales
  WHERE year = 2025
    AND month = 10
    AND date IS NOT NULL
),
agg AS (
  SELECT dte, SUM(sale_dollars) AS sales_usd
  FROM d
  GROUP BY dte
),
lim AS (SELECT MAX(dte) AS max_dte FROM agg)
SELECT
  a.dte,
  a.sales_usd,
  LAG(a.sales_usd) OVER (ORDER BY a.dte) AS prev_day_sales,
  ROUND(
    (a.sales_usd - LAG(a.sales_usd) OVER (ORDER BY a.dte))
    / NULLIF(LAG(a.sales_usd) OVER (ORDER BY a.dte), 0) * 100, 2
  ) AS dod_growth_pct
FROM agg a
CROSS JOIN lim
WHERE a.dte >= date_sub(lim.max_dte, 30)
ORDER BY a.dte;


dte,sales_usd,prev_day_sales,dod_growth_pct
2012-10-17,14992.199999999988,null,null


In [0]:
%sql
SELECT
  CAST(date AS DATE) AS dte,
  SUM(sale_dollars)  AS sales_usd
FROM gold.fact_sales
WHERE year = 2025 AND month = 10 AND date IS NOT NULL
GROUP BY CAST(date AS DATE)
ORDER BY dte;


dte,sales_usd
2012-02-29,15920.099999999935
2012-07-12,14401.199999999988
2012-07-30,1678.0999999999979
2012-10-17,14992.199999999988


In [0]:
%sql
-- Diagnóstico: ¿la fecha interna no coincide con la partición?
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

SELECT
  year,
  month,
  COUNT(*)              AS rows,
  MIN(TRY_TO_DATE(date)) AS min_date,
  MAX(TRY_TO_DATE(date)) AS max_date
FROM fact_sales
GROUP BY year, month
ORDER BY year, month;


year,month,rows,min_date,max_date
2025,10,910,2012-02-29,2012-10-17


In [0]:
%sql
-- Diagnóstico: ¿la fecha de la venta coincide con la partición year/month?
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH f AS (
  SELECT
    /* convierte la columna de fecha de venta a DATE */
    TRY_CAST(date AS DATE) AS sale_dt,
    year,
    month
  FROM fact_sales
)
SELECT
  year,
  month,
  COUNT(*)                                   AS rows,
  MIN(sale_dt)                               AS min_date,
  MAX(sale_dt)                               AS max_date,
  /* cuántas filas tienen desfasaje entre sale_dt y year/month */
  SUM(CASE WHEN sale_dt IS NOT NULL AND (YEAR(sale_dt) <> year OR MONTH(sale_dt) <> month)
           THEN 1 ELSE 0 END)                AS mismatched_rows
FROM f
GROUP BY year, month
ORDER BY year, month;


year,month,rows,min_date,max_date,mismatched_rows
2025,10,910,2012-02-29,2012-10-17,910


In [0]:
%sql
-- Muestras de filas con desfasaje entre la fecha real y la partición year/month
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

SELECT
  invoice_line_no,
  TRY_CAST(date AS DATE) AS sale_dt,
  year,
  month,
  store           AS store_id,
  itemno          AS item_no,
  sale_bottles,
  sale_dollars
FROM fact_sales
WHERE TRY_CAST(date AS DATE) IS NOT NULL
  AND (YEAR(TRY_CAST(date AS DATE)) <> year OR MONTH(TRY_CAST(date AS DATE)) <> month)
ORDER BY sale_dt
LIMIT 50;


invoice_line_no,sale_dt,year,month,store_id,item_no,sale_bottles,sale_dollars
381800024,2012-02-29,2025,10,3818,10627,3,43.68
381800047,2012-02-29,2025,10,3818,41746,1,11.94
381800108,2012-02-29,2025,10,3818,11773,8,18.72
381800063,2012-02-29,2025,10,3818,45247,2,13.00
381800093,2012-02-29,2025,10,3818,64996,1,17.27
381800022,2012-02-29,2025,10,3818,26826,2,39.02
381800011,2012-02-29,2025,10,3818,25606,4,39.16
381800105,2012-02-29,2025,10,3818,43333,3,9.00
381800003,2012-02-29,2025,10,3818,35315,12,60.60
381800004,2012-02-29,2025,10,3818,35918,6,62.28


In [0]:
%sql
-- Rebuild GOLD con year/month calculados desde la fecha real
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

CREATE OR REPLACE TABLE fact_sales
USING DELTA
PARTITIONED BY (year, month)
AS
SELECT
  invoice_line_no,
  TRY_CAST(date AS DATE)                   AS sale_date,
  store,
  name,
  address,
  city,
  zipcode,
  county_number,
  county,
  category,
  category_name,
  vendor_no,
  vendor_name,
  itemno,
  im_desc                                   AS item_desc,     -- <- corrección
  COALESCE(pack_int, pack)                  AS pack,          -- usa pack_int si existe
  bottle_volume_ml,
  state_bottle_cost,
  state_bottle_retail,
  sale_bottles,
  sale_dollars,
  sale_liters,
  sale_gallons,
  YEAR(TRY_CAST(date AS DATE))              AS year,          -- particiones correctas
  MONTH(TRY_CAST(date AS DATE))             AS month,
  current_timestamp()                       AS loaded_at
FROM ct_andresolguin_finalproject.silver.iowa_clean_strict
WHERE TRY_CAST(date AS DATE) IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Verificación de la tabla GOLD recién reconstruida
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

SELECT
  year,
  month,
  COUNT(*)                               AS rows,
  MIN(sale_date)                         AS min_date,
  MAX(sale_date)                         AS max_date,
  SUM(CASE WHEN sale_date IS NOT NULL
            AND (YEAR(sale_date) <> year OR MONTH(sale_date) <> month)
           THEN 1 ELSE 0 END)            AS mismatched_rows
FROM fact_sales
GROUP BY year, month
ORDER BY year, month;


year,month,rows,min_date,max_date,mismatched_rows
2012,2,620,2012-02-29,2012-02-29,0
2012,7,190,2012-07-12,2012-07-30,0
2012,10,100,2012-10-17,2012-10-17,0


In [0]:
%sql
-- Paso 3 — Rebuild de GOLD desde la silver estricta
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

CREATE OR REPLACE TABLE fact_sales
USING DELTA
PARTITIONED BY (year, month)
AS
SELECT *
FROM ct_andresolguin_finalproject.silver.iowa_clean_v2_strict;


num_affected_rows,num_inserted_rows


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

SELECT 
  COUNT(*)                               AS rows,
  MIN(sale_date)                         AS min_date,
  MAX(sale_date)                         AS max_date,
  COUNT(DISTINCT year)                   AS distinct_years,
  COUNT(DISTINCT make_date(year,month,1)) AS distinct_year_months,
  SUM(CASE 
        WHEN sale_date IS NOT NULL 
         AND (YEAR(sale_date) <> year OR MONTH(sale_date) <> month) 
        THEN 1 ELSE 0 
      END)                               AS mismatched_rows
FROM fact_sales;


rows,min_date,max_date,distinct_years,distinct_year_months,mismatched_rows
108224000,2012-01-03,2025-05-23,14,143,0


In [0]:
%sql
SELECT COUNT(*) AS rows, MIN(sale_date) AS min_date, MAX(sale_date) AS max_date
FROM fact_sales;


rows,min_date,max_date
108224000,2012-01-03,2025-05-23


In [0]:
%sql
-- Tendencia diaria (últimos 30 días) desde GOLD
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH d AS (
  SELECT CAST(sale_date AS DATE) AS dte, sale_dollars
  FROM fact_sales
  WHERE sale_date IS NOT NULL
),
lim AS (SELECT MAX(dte) AS max_dte FROM d),
last_30 AS (
  SELECT d.*
  FROM d JOIN lim ON 1=1
  WHERE d.dte >= date_sub(lim.max_dte, 30)
),
agg AS (
  SELECT dte, SUM(sale_dollars) AS sales_usd
  FROM last_30
  GROUP BY dte
)
SELECT
  dte,
  sales_usd,
  LAG(sales_usd) OVER (ORDER BY dte) AS prev_day_sales,
  ROUND(
    (sales_usd - LAG(sales_usd) OVER (ORDER BY dte))
    / NULLIF(LAG(sales_usd) OVER (ORDER BY dte), 0) * 100, 2
  ) AS dod_growth_pct
FROM agg
ORDER BY dte;


dte,sales_usd,prev_day_sales,dod_growth_pct
2025-05-01,180.00,null,null
2025-05-14,993.60,180.00,452.00
2025-05-23,2070.00,993.60,108.33


In [0]:
%sql
-- Tendencia mensual con YoY, excluyendo el mes en curso
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH d AS (
  SELECT CAST(sale_date AS DATE) AS dte, sale_dollars
  FROM fact_sales
  WHERE sale_date IS NOT NULL
),
lim AS (
  SELECT date_trunc('month', MAX(dte)) AS max_month_start
  FROM d
),
m AS (
  SELECT
    date_trunc('month', dte)                AS month_start,
    YEAR(dte)                               AS year,
    MONTH(dte)                              AS month,
    SUM(sale_dollars)                       AS sales_usd
  FROM d, lim
  WHERE date_trunc('month', d.dte) < lim.max_month_start   -- evita mes incompleto
  GROUP BY date_trunc('month', dte), YEAR(dte), MONTH(dte)
)
SELECT
  year,
  month,
  sales_usd,
  LAG(sales_usd, 1) OVER (ORDER BY month_start)           AS prev_month_sales,
  ROUND(
    (sales_usd - LAG(sales_usd, 1) OVER (ORDER BY month_start))
    / NULLIF(LAG(sales_usd, 1) OVER (ORDER BY month_start), 0) * 100, 2
  )                                                       AS mom_growth_pct,
  LAG(sales_usd, 12) OVER (ORDER BY month_start)          AS prev_year_same_month_sales,
  ROUND(
    (sales_usd - LAG(sales_usd, 12) OVER (ORDER BY month_start))
    / NULLIF(LAG(sales_usd, 12) OVER (ORDER BY month_start), 0) * 100, 2
  )                                                       AS yoy_growth_pct
FROM m
ORDER BY year, month;


year,month,sales_usd,prev_month_sales,mom_growth_pct,prev_year_same_month_sales,yoy_growth_pct
2012,1,67086411.70,null,null,null,null
2012,2,73463790.50,67086411.70,9.51,null,null
2012,3,75757663.00,73463790.50,3.12,null,null
2012,4,82528779.80,75757663.00,8.94,null,null
2012,5,92514206.00,82528779.80,12.10,null,null
2012,6,86280603.90,92514206.00,-6.74,null,null
2012,7,86884157.40,86280603.90,0.70,null,null
2012,8,90406339.70,86884157.40,4.05,null,null
2012,9,74616599.10,90406339.70,-17.47,null,null
2012,10,127867806.30,74616599.10,71.37,null,null


In [0]:
%sql
-- Tendencia ANUAL con YoY excluyendo el año en curso (incompleto)
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH y AS (
  SELECT
    YEAR(sale_date)         AS year,
    SUM(sale_dollars)       AS sales_usd
  FROM fact_sales
  WHERE sale_date IS NOT NULL
  GROUP BY YEAR(sale_date)
)
SELECT
  y.year,
  y.sales_usd,
  LAG(y.sales_usd) OVER (ORDER BY y.year) AS prev_year_sales,
  ROUND(
    (y.sales_usd - LAG(y.sales_usd) OVER (ORDER BY y.year))
    / NULLIF(LAG(y.sales_usd) OVER (ORDER BY y.year), 0) * 100, 2
  ) AS yoy_growth_pct
FROM y
WHERE y.year < YEAR(current_date())     -- <--- filtra el año en curso
ORDER BY y.year;


year,sales_usd,prev_year_sales,yoy_growth_pct
2012,1044910768.50,null,null
2013,1056612607.70,1044910768.50,1.12
2014,1110593742.20,1056612607.70,5.11
2015,1162610784.80,1110593742.20,4.68
2016,1355524366.80,1162610784.80,16.59
2017,1671837722.50,1355524366.80,23.34
2018,1737609307.90,1671837722.50,3.93
2019,1439194938.30,1737609307.90,-17.17
2020,1437628010.70,1439194938.30,-0.11
2021,1569475948.80,1437628010.70,9.17


In [0]:
%sql
-- Catálogo y esquema
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

-- Top 20 tiendas por ventas (sale_dollars) - histórico completo
SELECT
  store_id,
  store_name,
  county,
  city,
  SUM(sale_bottles) AS bottles,
  SUM(sale_dollars) AS sales_usd
FROM fact_sales
GROUP BY store_id, store_name, county, city
HAVING SUM(sale_dollars) > 0
ORDER BY sales_usd DESC
LIMIT 20;


store_id,store_name,county,city,bottles,sales_usd
4829,CENTRAL CITY 2,POLK,DES MOINES,23265520,403784298.80
2633,HY-VEE #3 / BDI / DES MOINES,POLK,DES MOINES,18796270,332153183.10
2512,HY-VEE WINE AND SPIRITS / IOWA CITY,JOHNSON,IOWA CITY,12466560,189372410.20
3385,SAM'S CLUB 8162 / CEDAR RAPIDS,LINN,CEDAR RAPIDS,8454040,147173550.20
3952,LOT-A-SPIRITS,SCOTT,BETTENDORF,7980910,136380688.50
3773,BENZ DISTRIBUTING,LINN,CEDAR RAPIDS,7221020,127565200.40
3420,SAM'S CLUB 6344 / WINDSOR HEIGHTS,POLK,WINDSOR HEIGHTS,6158590,112884781.50
2190,"CENTRAL CITY LIQUOR, INC.",POLK,DES MOINES,7040600,106085219.90
3354,SAM'S CLUB 8238 / DAVENPORT,SCOTT,DAVENPORT,5859640,105092043.50
5102,WILKIE LIQUORS,LINN,MOUNT VERNON,6774040,103404878.30


In [0]:
%sql
-- Catálogo y esquema
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

-- Ranking de CONDADOS (Top 20 por ventas históricas)
SELECT
  county,
  COUNT(DISTINCT store_id)         AS stores,      -- cuántas tiendas operaron en el condado
  SUM(sale_bottles)                AS bottles,
  SUM(sale_dollars)                AS sales_usd
FROM fact_sales
GROUP BY county
HAVING SUM(sale_dollars) > 0
ORDER BY sales_usd DESC
LIMIT 20;


county,stores,bottles,sales_usd
POLK,388,236102770,3150399318.70
LINN,212,99235380,1255438697.40
SCOTT,136,85383400,1047129428.50
JOHNSON,110,58212230,850740548.50
BLACK HAWK,170,70112140,820814227.00
POTTAWATTAMIE,97,41052560,519607604.50
WOODBURY,72,39442300,506426909.50
DUBUQUE,82,32508330,440838884.70
STORY,77,31074020,439765408.80
CERRO GORDO,46,23503250,306575998.20


In [0]:
%sql
-- Catálogo y esquema
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

-- Top 20 categorías por ventas históricas
SELECT
  category_name,
  SUM(sale_bottles) AS bottles,
  SUM(sale_dollars) AS sales_usd
FROM fact_sales
GROUP BY category_name
HAVING SUM(sale_dollars) > 0
ORDER BY sales_usd DESC
LIMIT 20;


category_name,bottles,sales_usd
CANADIAN WHISKIES,117216070,1651825373.80
AMERICAN VODKAS,141184240,1292851700.30
SPICED RUM,65307170,977654495.00
STRAIGHT BOURBON WHISKIES,51524510,947233376.30
IMPORTED VODKAS,37844440,728665886.90
WHISKEY LIQUEUR,80941330,685938469.10
TENNESSEE WHISKIES,27264660,598612472.90
VODKA 80 PROOF,73560770,585124665.20
BLENDED WHISKIES,42182330,426320764.10
100% AGAVE TEQUILA,14746470,400144528.80


In [0]:
%sql
-- Categorías con mayor crecimiento (%YoY) en el último año completo (menos restrictivo)
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH y AS (
  SELECT
    category_name,
    YEAR(sale_date) AS yr,
    SUM(sale_dollars) AS sales_usd
  FROM fact_sales
  GROUP BY category_name, YEAR(sale_date)
),
mx AS (SELECT MAX(yr) AS max_yr FROM y),
ref AS (SELECT max_yr - 1 AS ref_year FROM mx),  -- último año completo
w AS (
  SELECT
    category_name,
    yr,
    sales_usd,
    LAG(sales_usd) OVER (PARTITION BY category_name ORDER BY yr) AS prev_year_sales
  FROM y
)
SELECT
  w.category_name,
  r.ref_year,
  w.sales_usd,
  w.prev_year_sales,
  ROUND((w.sales_usd - w.prev_year_sales) / NULLIF(w.prev_year_sales, 0) * 100, 2) AS yoy_growth_pct
FROM w
JOIN ref r ON w.yr = r.ref_year
WHERE w.sales_usd >= 100000
  AND w.prev_year_sales >= 100000
ORDER BY yoy_growth_pct DESC
LIMIT 20;


category_name,ref_year,sales_usd,prev_year_sales,yoy_growth_pct
AMERICAN VODKAS,2024,958362.80,1120660.50,-14.48
MIXTO TEQUILA,2024,150536.20,197962.00,-23.96
IMPORTED VODKAS,2024,128061.50,170369.90,-24.83
SPICED RUM,2024,209956.60,322105.40,-34.82
BLENDED WHISKIES,2024,134746.00,208507.60,-35.38
STRAIGHT BOURBON WHISKIES,2024,441926.70,731702.10,-39.60
TEMPORARY & SPECIALTY PACKAGES,2024,142660.90,237926.50,-40.04
IMPORTED CORDIALS & LIQUEURS,2024,196448.80,366145.70,-46.35
TENNESSEE WHISKIES,2024,200734.00,422576.30,-52.50
CANADIAN WHISKIES,2024,469077.00,1004284.10,-53.29


In [0]:
%sql
-- Catálogo y esquema
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH d AS (
  SELECT
    fs.item_no,
    di.item_desc,
    /* ingresos y costos por fila */
    COALESCE(fs.sale_dollars, fs.state_bottle_retail * fs.sale_bottles) AS revenue_usd_row,
    (fs.state_bottle_cost * fs.sale_bottles)                             AS cost_usd_row,
    fs.sale_bottles
  FROM fact_sales fs
  JOIN ct_andresolguin_finalproject.gold.dim_item di
    ON fs.item_no = di.item_no
  -- WHERE fs.sale_date >= DATE '2024-01-01'    -- (opcional para acotar periodo)
)
SELECT
  item_no,
  item_desc,
  SUM(sale_bottles)                               AS bottles,
  SUM(revenue_usd_row)                            AS revenue_usd,
  SUM(cost_usd_row)                               AS cost_usd,
  SUM(revenue_usd_row) - SUM(cost_usd_row)        AS margin_usd,
  ROUND( (SUM(revenue_usd_row) - SUM(cost_usd_row))
         / NULLIF(SUM(revenue_usd_row),0) * 100, 2) AS margin_pct
FROM d
GROUP BY item_no, item_desc
HAVING SUM(sale_bottles) > 0 AND SUM(revenue_usd_row) > 0
ORDER BY margin_usd DESC
LIMIT 20;


item_no,item_desc,bottles,revenue_usd,cost_usd,margin_usd,margin_pct
11788,BLACK VELVET,18674900,288676889.20,191675310.00,97001579.20,33.60
43337,CAPTAIN MORGAN ORIGINAL SPICED,12996410,227990119.10,151949263.50,76040855.60,33.35
38178,TITOS HANDMADE VODKA,6407130,181545463.90,121511851.80,60033612.10,33.07
11297,CROWN ROYAL,6407820,179679021.20,119800066.00,59878955.20,33.33
26827,JACK DANIELS OLD #7 BLACK LABEL,5966760,164041139.10,109361287.70,54679851.40,33.33
88296,PATRON SILVER,3748280,153464054.50,102594055.00,50869999.50,33.15
11296,CROWN ROYAL,6338590,146002602.40,97664494.30,48338108.10,33.11
36308,HAWKEYE VODKA,13104690,140943419.20,93890789.50,47052629.70,33.38
38176,TITOS HANDMADE VODKA,9506400,139595623.20,93008190.60,46587432.60,33.37
43338,CAPTAIN MORGAN ORIGINAL SPICED,4964710,133234408.20,88713798.70,44520609.50,33.42


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH d AS (
  SELECT
    di.category_name,
    COALESCE(fs.sale_dollars, fs.state_bottle_retail * fs.sale_bottles) AS revenue_usd_row,
    (fs.state_bottle_cost * fs.sale_bottles)                             AS cost_usd_row,
    fs.sale_bottles
  FROM fact_sales fs
  JOIN ct_andresolguin_finalproject.gold.dim_item di
    ON fs.item_no = di.item_no
)
SELECT
  category_name,
  SUM(sale_bottles)                               AS bottles,
  SUM(revenue_usd_row)                            AS revenue_usd,
  SUM(cost_usd_row)                               AS cost_usd,
  SUM(revenue_usd_row) - SUM(cost_usd_row)        AS margin_usd,
  ROUND( (SUM(revenue_usd_row) - SUM(cost_usd_row))
         / NULLIF(SUM(revenue_usd_row),0) * 100, 2) AS margin_pct
FROM d
GROUP BY category_name
HAVING SUM(sale_bottles) > 0 AND SUM(revenue_usd_row) > 0
ORDER BY margin_usd DESC
LIMIT 20;


category_name,bottles,revenue_usd,cost_usd,margin_usd,margin_pct
AMERICAN VODKAS,224352660,2012361300.40,1340758481.70,671602818.70,33.37
CANADIAN WHISKIES,117933850,1671442580.50,1113780682.70,557661897.80,33.36
SPICED RUM,65207660,976462687.20,650288829.90,326173857.30,33.40
STRAIGHT BOURBON WHISKIES,49716690,921434512.50,614574437.80,306860074.70,33.30
WHISKEY LIQUEUR,90762260,701252786.80,466757848.40,234494938.40,33.44
IMPORTED VODKAS,31059580,615516855.10,408855952.50,206660902.60,33.58
TENNESSEE WHISKIES,27263490,599678567.80,399967237.00,199711330.80,33.30
100% AGAVE TEQUILA,19694190,532095274.40,355351318.80,176743955.60,33.22
AMERICAN FLAVORED VODKA,43535200,462518262.20,307836112.40,154682149.80,33.44
BLENDED WHISKIES,43160020,440332336.10,293508122.70,146824213.40,33.34


In [0]:
%sql
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH d AS (
  SELECT
    di.vendor_name,
    COALESCE(fs.sale_dollars, fs.state_bottle_retail * fs.sale_bottles) AS revenue_usd_row,
    (fs.state_bottle_cost * fs.sale_bottles)                             AS cost_usd_row,
    fs.sale_bottles
  FROM fact_sales fs
  JOIN ct_andresolguin_finalproject.gold.dim_item di
    ON fs.item_no = di.item_no
)
SELECT
  vendor_name,
  SUM(sale_bottles)                               AS bottles,
  SUM(revenue_usd_row)                            AS revenue_usd,
  SUM(cost_usd_row)                               AS cost_usd,
  SUM(revenue_usd_row) - SUM(cost_usd_row)        AS margin_usd,
  ROUND( (SUM(revenue_usd_row) - SUM(cost_usd_row))
         / NULLIF(SUM(revenue_usd_row),0) * 100, 2) AS margin_pct
FROM d
GROUP BY vendor_name
HAVING SUM(sale_bottles) > 0 AND SUM(revenue_usd_row) > 0
ORDER BY margin_usd DESC
LIMIT 20;


vendor_name,bottles,revenue_usd,cost_usd,margin_usd,margin_pct
DIAGEO AMERICAS,165727220,3015818735.40,2010445516.50,1005373218.90,33.34
SAZERAC COMPANY INC,213103310,1900238649.70,1264699092.10,635539557.60,33.45
JIM BEAM BRANDS,76982220,1090824426.60,726177236.90,364647189.70,33.43
PERNOD RICARD USA,58167610,1010861359.10,673723242.80,337138116.30,33.35
HEAVEN HILL BRANDS,94782940,987729067.50,656736384.80,330992682.70,33.51
BACARDI USA INC,43210160,868672679.40,578252240.50,290420438.90,33.43
LUXCO INC,100714040,795585112.50,531735275.10,263849837.40,33.16
BROWN FORMAN CORP.,31230880,704568027.60,470116507.40,234451520.20,33.28
PROXIMO,34556570,512708502.50,341920174.00,170788328.50,33.31
FIFTH GENERATION INC,25395770,474062071.90,317270340.30,156791731.60,33.07


In [0]:
%sql
-- PASO 1 — Panel mensual por ARTÍCULO: precio unitario vs volumen
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

CREATE OR REPLACE TEMP VIEW item_month_panel AS
WITH m AS (
  SELECT
    date_trunc('month', sale_date)                     AS month,
    item_no,
    item_desc,
    category_name,
    -- volumen y facturación
    SUM(sale_bottles)                                  AS qty_bottles,
    SUM(sale_dollars)                                  AS revenue_usd,
    -- precio unitario efectivo (evita /0)
    SUM(sale_dollars) / NULLIF(SUM(sale_bottles), 0)   AS unit_price_usd,
    -- referencia: retail promedio reportado (siempre >0 para control)
    AVG(NULLIF(state_bottle_retail,0))                 AS avg_reported_retail_usd
  FROM fact_sales
  WHERE sale_date IS NOT NULL
  GROUP BY 1,2,3,4
)
SELECT *
FROM m
WHERE qty_bottles > 0
  AND unit_price_usd > 0;



## Cierre — Validaciones y KPIs rápidos
- Cuenta de filas, `min(sale_date)`, `max(sale_date)`.  
- Verificación de particiones (`year, month`).  
- Devuelve estado `OK`.


In [0]:
%sql
-- PASO 2 (robusto) — Elasticidad precio–volumen por ARTÍCULO + sugerencia
USE CATALOG ct_andresolguin_finalproject;
USE SCHEMA gold;

WITH b AS (       -- base log-log
  SELECT
    month,
    item_no,
    MAX(item_desc)          AS item_desc,
    MAX(category_name)      AS category_name,
    qty_bottles,
    revenue_usd,
    unit_price_usd,
    LOG(unit_price_usd)     AS ln_p,
    LOG(qty_bottles)        AS ln_q
  FROM item_month_panel
  GROUP BY month, item_no, qty_bottles, revenue_usd, unit_price_usd
),

mx AS (           -- última ventana por ítem
  SELECT item_no, MAX(month) AS max_m
  FROM b
  GROUP BY item_no
),

recent AS (       -- promedio últimos 3 meses
  SELECT
    b.item_no,
    AVG(b.unit_price_usd)  AS recent_price_usd,
    AVG(b.qty_bottles)     AS recent_qty
  FROM b
  JOIN mx ON b.item_no = mx.item_no
  WHERE b.month >= add_months(mx.max_m, -2)
  GROUP BY b.item_no
),

est AS (          -- estimación de elasticidad y calidad de señal
  SELECT
    item_no,
    MAX(item_desc)                                AS item_desc,
    MAX(category_name)                            AS category_name,
    COUNT(*)                                      AS n_months,
    regr_slope(ln_q, ln_p)                        AS elasticity,      -- pendiente
    regr_r2(ln_q, ln_p)                           AS r2,              -- coef. de determinación
    CASE
      WHEN regr_r2(ln_q, ln_p) IS NULL THEN NULL
      ELSE SIGN(regr_slope(ln_q, ln_p)) * SQRT(regr_r2(ln_q, ln_p))
    END                                           AS corr_signed,     -- “corr” con signo, sin /0
    AVG(unit_price_usd)                           AS avg_price_usd,
    AVG(qty_bottles)                              AS avg_qty,
    COUNT(DISTINCT unit_price_usd)                AS n_prices_distinct,
    COUNT(DISTINCT qty_bottles)                   AS n_qty_distinct
  FROM b
  GROUP BY item_no
)

SELECT
  e.item_no,
  e.item_desc,
  e.category_name,
  e.n_months,
  e.elasticity,
  e.r2,
  e.corr_signed,
  r.recent_price_usd,
  r.recent_qty,
  CASE
    WHEN e.n_months >= 6
         AND COALESCE(ABS(e.corr_signed),0) >= 0.30
         AND e.elasticity <= -1.0
      THEN 'Bajar precio 5% (demanda elástica)'
    WHEN e.n_months >= 6
         AND COALESCE(ABS(e.corr_signed),0) >= 0.30
         AND e.elasticity < 0
      THEN 'Subir precio 3% (demanda inelástica)'
    WHEN e.n_months >= 6
         AND COALESCE(ABS(e.corr_signed),0) >= 0.30
         AND e.elasticity >= 0
      THEN 'Sin ajuste (pendiente ≥ 0)'
    ELSE 'No concluyente (pocos datos, varianza 0 o baja señal)'
  END AS price_action,
  CASE
    WHEN e.n_months >= 6
         AND COALESCE(ABS(e.corr_signed),0) >= 0.30
         AND e.elasticity <= -1.0
      THEN ROUND(r.recent_price_usd * 0.95, 2)
    WHEN e.n_months >= 6
         AND COALESCE(ABS(e.corr_signed),0) >= 0.30
         AND e.elasticity < 0
      THEN ROUND(r.recent_price_usd * 1.03, 2)
    ELSE NULL
  END AS suggested_price_usd
FROM est e
LEFT JOIN recent r ON r.item_no = e.item_no
-- señal mínima y evitar casos degenerados (todos los precios o cantidades iguales)
WHERE e.n_months >= 4
  AND e.n_prices_distinct >= 2
  AND e.n_qty_distinct    >= 2
ORDER BY
  CASE WHEN e.n_months >= 6 AND COALESCE(ABS(e.corr_signed),0) >= 0.30 THEN 0 ELSE 1 END,
  e.elasticity ASC
LIMIT 100;


item_no,item_desc,category_name,n_months,elasticity,r2,corr_signed,recent_price_usd,recent_qty,price_action,suggested_price_usd
4810,CRAIGELLACHIE 13YR SINGLE MALT,TEMPORARY & SPECIALTY PACKAGES,12,-9903.749277663204,0.335916761398827,-0.5795832652853488,41.2333333333333333330000,120.0,Bajar precio 5% (demanda elástica),39.17
64915,CIROC MANGO,IMPORTED FLAVORED VODKA,7,-7130.798772302081,0.31269218543935684,-0.5591888638370375,34.1285714285714285713333,90.0,Bajar precio 5% (demanda elástica),32.42
46142,CALIBER SILVER RUM,WHITE RUM,67,-3279.1820949789085,0.15774701620742257,-0.39717378590161584,11.0400000000000000000000,60.0,Bajar precio 5% (demanda elástica),10.49
27290,IOWA SHINE,BLENDED WHISKIES,16,-2738.6124390092873,0.37659334143397083,-0.6136720145435759,15.1400000000000000000000,40.0,Bajar precio 5% (demanda elástica),14.38
66107,OR-G,MISC. IMPORTED CORDIALS & LIQUEURS,38,-2392.7265816141103,0.324470313672945,-0.5696229574665552,18.7500000000000000000000,180.0,Bajar precio 5% (demanda elástica),17.81
64626,COURVOISIER GOLD,MISC. IMPORTED CORDIALS & LIQUEURS,13,-2304.8026145773506,0.1965981535120293,-0.44339390333204776,22.0466666666666666666667,123.33333333333333,Bajar precio 5% (demanda elástica),20.94
67194,X RATED FUSION LIQUEUR,MISC. IMPORTED CORDIALS & LIQUEURS,76,-1943.989234825085,0.1744859262394922,-0.4177151257011077,28.0100000000000000000000,156.66666666666666,Bajar precio 5% (demanda elástica),26.61
903230,STREGA LIQUORE,SPECIAL ORDER ITEMS,19,-1900.7740462479762,0.2917542213873732,-0.5401427787051987,28.2000000000000000000000,240.0,Bajar precio 5% (demanda elástica),26.79
11390,TAP 357,CANADIAN WHISKIES,25,-1874.8467191242319,0.13093275707304144,-0.3618463169261799,22.4400000000000000000000,80.0,Bajar precio 5% (demanda elástica),21.32
34138,CHAMBORD FLAVORED VODKA,IMPORTED VODKA - MISC,23,-1640.607710787909,0.2474492060672756,-0.497442666110654,18.7633333333333333333333,40.0,Bajar precio 5% (demanda elástica),17.83
